In [1]:
# stdlib
import os

# third party
import requests

# syft absolute
import syft as sy
from syft.client.domain_client import DomainClient
from syft.custom_worker.config import DockerWorkerConfig
from syft.service.request.request import Request
from syft.service.response import SyftSuccess
from syft.service.worker.worker_image import SyftWorkerImage
from syft.service.worker.worker_pool import SyftWorker
from syft.service.worker.worker_pool import WorkerPool

In [2]:
registry = "k3d-registry.localhost:5800"
repo = "openmined/grid-backend"

if "k3d" in registry:
    res = requests.get(url=f"http://{registry}/v2/{repo}/tags/list")
    tag = res.json()["tags"][0]
else:
    tag = sy.__version__

external_registry = os.getenv("EXTERNAL_REGISTRY", registry)
external_registry_username = os.getenv("EXTERNAL_REGISTRY_USERNAME", None)
external_registry_password = os.getenv("EXTERNAL_REGISTRY_PASSWORD", None)

In [3]:
def test():
    domain_client: DomainClient = sy.login(
        port=9082, email="info@openmined.org", password="changethis"
    )
    image_registry_list = domain_client.api.services.image_registry.get_all()
    if len(image_registry_list) > 1:
        raise Exception("Only one registry should be present for testing")

    elif len(image_registry_list) == 1:
        assert (
            image_registry_list[0].url == external_registry
        ), "External registry different from the one set in the environment variable"
        return image_registry_list[0].id
    else:
        registry_add_result = domain_client.api.services.image_registry.add(
            external_registry
        )

        assert isinstance(registry_add_result, sy.SyftSuccess), str(registry_add_result)

        image_registry_list = domain_client.api.services.image_registry.get_all()
        return image_registry_list[0].id

In [4]:
domain_client: DomainClient = sy.login(
    port=9082, email="info@openmined.org", password="changethis"
)

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [8]:
domain_client.api.services.image_registry.get_all()

[SyftImageRegistry(url=k3d-registry.localhost:5800)]

In [6]:
test() == test()

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

True

In [7]:
domain_client = sy.login(port=9082, email="info@openmined.org", password="changethis")

# Submit Docker Worker Config
docker_config_rl = f"""
    FROM {registry}/{repo}:{tag}
    RUN pip install recordlinkage
"""
docker_config = DockerWorkerConfig(dockerfile=docker_config_rl)

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [9]:
# Submit Worker Image
submit_result = domain_client.api.services.worker_image.submit_dockerfile(
    docker_config=docker_config
)
assert isinstance(submit_result, SyftSuccess)
assert len(domain_client.images.get_all()) == 2

In [10]:
# Validate if we can get the worker image object from its config
workerimage = domain_client.api.services.worker_image.get_by_config(docker_config)
assert not isinstance(workerimage, sy.SyftError)

In [11]:
# Build docker image
docker_tag = "openmined/custom-worker-rl:latest"
docker_build_result = domain_client.api.services.worker_image.build(
    image_uid=workerimage.id,
    tag=docker_tag,
    registry_uid=test(),
)
assert isinstance(docker_build_result, SyftSuccess)

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [12]:
# Refresh the worker image object
workerimage = domain_client.images.get_by_uid(workerimage.id)
assert not isinstance(workerimage, sy.SyftSuccess)

assert workerimage.is_built
assert workerimage.image_identifier is not None
assert workerimage.image_identifier.repo_with_tag == docker_tag
assert workerimage.image_hash is not None

In [14]:
domain_client: DomainClient = sy.login(
    port=9082, email="info@openmined.org", password="changethis"
)
assert len(domain_client.worker_pools.get_all()) == 1

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [15]:
# Submit Docker Worker Config
docker_config_opendp = f"""
    FROM {registry}/{repo}:{tag}
    RUN pip install opendp
"""
docker_config = DockerWorkerConfig(dockerfile=docker_config_opendp)

In [16]:
# Submit Worker Image
submit_result = domain_client.api.services.worker_image.submit_dockerfile(
    docker_config=docker_config
)
assert isinstance(submit_result, SyftSuccess)

In [17]:
worker_image = domain_client.api.services.worker_image.get_by_config(docker_config)
assert not isinstance(worker_image, sy.SyftError)
assert worker_image is not None
assert not worker_image.is_built

In [18]:
# Build docker image
docker_tag = "openmined/custom-worker-opendp:latest"
docker_build_result = domain_client.api.services.worker_image.build(
    image_uid=worker_image.id, tag=docker_tag, registry_uid=test()
)
assert isinstance(docker_build_result, SyftSuccess)

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [19]:
push_result = None
push_result = domain_client.api.services.worker_image.push(
    worker_image.id,
    username=external_registry_username,
    password=external_registry_password,
)
assert isinstance(push_result, sy.SyftSuccess), str(push_result)

In [26]:
# Launch a worker pool
worker_pool_name = "custom-worker-pool-ver-1"
worker_pool_res = domain_client.api.services.worker_pool.launch(
    name=worker_pool_name,
    image_uid=worker_image.id,
    num_workers=3,
)
assert len(worker_pool_res) == 3

assert all(worker.error is None for worker in worker_pool_res)
assert len(domain_client.worker_pools.get_all()) == 2

In [28]:
worker_pool_res

[ContainerSpawnStatus(worker_name='custom-worker-pool-ver-1-0', worker=syft.service.worker.worker_pool.SyftWorker, error=None),
 ContainerSpawnStatus(worker_name='custom-worker-pool-ver-1-1', worker=syft.service.worker.worker_pool.SyftWorker, error=None),
 ContainerSpawnStatus(worker_name='custom-worker-pool-ver-1-2', worker=syft.service.worker.worker_pool.SyftWorker, error=None)]

In [29]:
worker_pool = domain_client.worker_pools[worker_pool_name]
assert len(worker_pool.worker_list) == 3

workers = worker_pool.workers
assert len(workers) == 3

for worker in workers:
    assert worker.worker_pool_name == worker_pool_name
    assert worker.image.id == worker_image.id

assert len(worker_pool.healthy_workers) == 3

# Grab the first worker
first_worker = workers[0]

# Check worker Logs
logs = domain_client.api.services.worker.logs(uid=first_worker.id)
assert not isinstance(logs, sy.SyftError)

# Check for worker status
status_res = domain_client.api.services.worker.status(uid=first_worker.id)
assert not isinstance(status_res, sy.SyftError)
assert isinstance(status_res, tuple)

In [30]:
# Delete the pool's workers
for worker in worker_pool.workers:
    res = domain_client.api.services.worker.delete(uid=worker.id, force=True)
    assert isinstance(res, sy.SyftSuccess)

# TODO: delete the launched pool

In [32]:
# Clean the build images
delete_result = domain_client.api.services.worker_image.remove(uid=worker_image.id)
assert isinstance(delete_result, sy.SyftSuccess)

AssertionError: 

In [38]:
domain_client: DomainClient = sy.login(
    port=9082, email="info@openmined.org", password="changethis"
)

ds_username = "sheldon"
ds_email = ds_username + "@example.com"
res = domain_client.register(
    name=ds_username,
    email=ds_email,
    password="secret_pw",
    password_verify="secret_pw",
)
# assert isinstance(res, SyftSuccess)
ds_client = sy.login(email=ds_email, password="secret_pw", port=9082)

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <syft-dev-node: High side Domain> as <sheldon@example.com>


In [41]:
# the DS makes a request to create an image and a pool based on the image
docker_config_np = f"""
    FROM {registry}/{repo}:{tag}
    RUN pip install numpy
"""
docker_config = DockerWorkerConfig(dockerfile=docker_config_np)
docker_tag = "openmined/custom-worker-np:latest"
worker_pool_name = "custom-worker-pool-numpy"
request = ds_client.api.services.worker_pool.create_image_and_pool_request(
    pool_name=worker_pool_name,
    num_workers=1,
    tag=docker_tag,
    config=docker_config,
    reason="I want to do some more cool data science with PySyft and Recordlinkage",
    registry_uid=test(),
)
assert isinstance(request, Request)
assert len(request.changes) == 2
assert request.changes[0].config == docker_config
assert request.changes[1].num_workers == 1
assert request.changes[1].pool_name == worker_pool_name

Logged into <syft-dev-node: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [43]:
# the domain client approve the request, so the image should be built
# and the worker pool should be launched
for r in domain_client.requests:
    if r.id == request.id:
        req_result = r.approve()
        break
assert isinstance(req_result, SyftSuccess)

Approving request for domain syft-dev-node


In [44]:
launched_pool = ds_client.api.services.worker_pool.get_by_name(worker_pool_name)
assert isinstance(launched_pool, WorkerPool)
assert launched_pool.name == worker_pool_name
assert len(launched_pool.worker_list) == 1

In [50]:
ds_client.api.services.worker_pool[2]

syft.service.worker.worker_pool.WorkerPool

In [51]:
worker: SyftWorker = launched_pool.workers[0]
assert launched_pool.name in worker.name
assert worker.status.value == "Running"
assert worker.healthcheck.value == "✅"
# assert worker.consumer_state.value == "Idle"
assert isinstance(worker.logs, str)
assert worker.job_id is None

In [52]:
built_image = ds_client.api.services.worker_image.get_by_config(docker_config)
assert isinstance(built_image, SyftWorkerImage)
assert built_image.id == launched_pool.image.id
assert worker.image.id == built_image.id

In [54]:
# third party
import numpy as np

In [55]:
# Dataset
data = np.array([1, 2, 3])
data_action_obj = sy.ActionObject.from_obj(data)
data_pointer = domain_client.api.services.action.set(data_action_obj)

# Function


@sy.syft_function(
    input_policy=sy.ExactMatch(x=data_pointer),
    output_policy=sy.SingleExecutionExactOutput(),
    worker_pool_name=launched_pool.name,
)
def custom_worker_func(x):
    return {"y": x + 1}


assert custom_worker_func.worker_pool_name == launched_pool.name
# Request code execution

SyftSuccess: Syft function 'custom_worker_func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [56]:
code_request = ds_client.code.request_code_execution(custom_worker_func)
assert isinstance(code_request, Request)
assert code_request.status.value == 0  # pending
for r in domain_client.requests:
    if r.id == code_request.id:
        code_req_result = r.approve(approve_nested=True)
        break
assert isinstance(code_req_result, SyftSuccess)

Approving request for domain syft-dev-node


In [57]:
job = ds_client.code.custom_worker_func(x=data_pointer, blocking=False)
assert job.status.value == "created"
job.wait()
assert job.status.value == "completed"

job = domain_client.jobs[-1]
assert job.job_worker_id == worker.id

# Validate the result received from the syft function
result = job.wait().get()
result_matches = result["y"] == data + 1
assert result_matches.all()

SyftWarning: This is a placeholder object, the real data lives on a different node and is not synced.

In [58]:
# Delete the workers of the launched pools
for worker in launched_pool.workers:
    res = domain_client.api.services.worker.delete(uid=worker.id, force=True)
    assert isinstance(res, sy.SyftSuccess)

# TODO: delete the launched pool

In [60]:
# Clean the build images

delete_result = domain_client.api.services.worker_image.remove(uid=built_image.id)
assert isinstance(delete_result, sy.SyftSuccess)

AssertionError: 